In [33]:
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:
# cd /content/drive/MyDrive/

In [35]:
# !pip install fastText

In [36]:
import fasttext

In [37]:
import json
import pandas as pd

def load_jsonl(fname):
    fin = open(fname, encoding="utf-8")
    data = []
    for line in fin:
        d = json.loads(line.strip())
        data.append(d)

    return data

def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as fo:
        for idx, d in enumerate(data):
            fo.write(json.dumps(d, ensure_ascii=False))
            fo.write("\n")

In [38]:
from itertools import groupby

def norm_word(word):
    groups = [list(s) for _, s in groupby(word)]
    ch = []
    extraToken = ""
    for g in groups:
        if len(g)>=3:
            
            if g[0]=="5":
              extraToken = "<lol>"
            else:
              extraToken = "<rep>"
            ch.append(g[0])  
        else:
            ch += g
    word = "".join(ch)+extraToken
    return word

In [39]:
DIR = "Datasets/"
MD = load_jsonl(f"{DIR}/../train_mispelling_dection.jsonl")[0]

### VISTEC-TP-TH-2021

In [40]:
from bs4 import BeautifulSoup
from tqdm import tqdm

sent = []

def ignore(m, c):
  if c.endswith(" ๆ"):
    return True

  if m.replace(" ", "")==c.replace(" ", ""):
    return True

  if ("ฯ" in c) and (m not in ["ๆลๆ", "พณฯท่าน", "ฯล"]):
    return True

  if ("." in m) or ("." in c):
    return True
  
  return False

def process_dom(dom):
  tokens = []
  cntMisp, cntIgnored = 0, 0
  if dom.name is None:
    words = str(dom).split("|")
    for w in words:
      w = w.strip()
      if len(w)==0:
        continue
      tokens.append((w, w))
  elif dom.name=="ne":
    words = dom.text.strip().split("|")
    for w in words:
      w = w.strip()
      if len(w)==0:
        continue
      tokens.append((w, w))

  elif dom.name=="msp":
    m = dom.text.replace("|", "").strip()
    c = dom["value"].strip()
    # assert("|" not in m)

    cntMisp += 1
    if ignore(m, c):
      tokens.append((m, m))
      cntIgnored += 1
    else:
      tokens.append((m, c))
      mispTokens.add((m, c))
  elif dom.name=="compound":
    for child in dom.children:
      tkn, cms, cig = process_dom(child)
      tokens += tkn
      cntMisp += cms
      cntIgnored += cig
  else:
    print(dom)
    raise(f"Unknown Tag: {dom.name}")

  return tokens, cntMisp, cntIgnored


mispTokens = set()
cntIgnored = 0
cntMisp = 0
cntToken = 0
with open(f"{DIR}/VISTEC-TP-TH-2021/train/VISTEC-TP-TH-2021_train_proprocessed.txt", encoding="utf-8") as fin:
  for line in tqdm(fin, total=40000):
    line = line.strip()
    s = BeautifulSoup("<div id='text'>"+line+"</div>")
    tokens = []
    for dom in s.find("div", {"id": "text"}).children:
      tkn, cms, cig = process_dom(dom)
      tokens += tkn
      cntMisp += cms
      cntIgnored += cig

      cntToken += len(tkn)
        
    sent.append(tokens)
    # misp.append(" ".join([t[0] for t in tokens]))
    # corr.append(" ".join([t[1] for t in tokens]))

print()
print("#Tokens:", cntToken)
print("#Misspelling Tokens:", cntMisp)
print("#Misspelling Tokens[Skip]:", cntIgnored, cntIgnored*100/cntMisp)


100%|██████████| 40000/40000 [00:13<00:00, 3016.19it/s]


#Tokens: 2697642
#Misspelling Tokens: 130779
#Misspelling Tokens[Skip]: 33418 25.553032214652198


In [41]:
with open(f"{DIR}/misp_VISTEC-TP-TH-2021_fasttext_training.txt", "w", encoding="utf-8") as fout:
    for tokens in sent:
      s = " ".join([t[0] for t in tokens])
      fout.write(s+"\n")

with open(f"{DIR}/corr_VISTEC-TP-TH-2021_fasttext_training.txt", "w", encoding="utf-8") as fout:
    for tokens in sent:
      s = " ".join([t[1] for t in tokens])
      fout.write(s+"\n")

In [42]:
with open(f"{DIR}/MST_VISTEC-TP-TH-2021_fasttext_training.txt", "w", encoding="utf-8") as fout:
    cnt = 0
    for tokens in sent:
      s = []
      for t in tokens:
        if t[1]!=t[0]:
          s.append(t[0])
          
          w = norm_word(t[0])
          if "<lol>" in w:
            s.append("<lol>")
          elif "<rep>" in w:
            s.append("<rep>")
          else:
            if w in MD:
              corr, mint = MD[w]
              if mint:
                s.append("<int>")
              else:
                s.append("<msp>")
            else:
              s.append("<msp>")
        else:
          s.append(t[0])
      # break
      s = " ".join(s)
      
      fout.write(s+"\n")

### Wisesight Train

In [46]:
wisesight = load_jsonl(f"{DIR}/WisesightSentiment/tokenized_train.jsonl")

In [48]:
with open(f"{DIR}/wisesight_train_fasttext_training.txt", "w", encoding="utf-8") as fout:
    for sent in wisesight:
      s = " ".join(sent["tokenized"])
      fout.write(s+"\n")

cntToken = 0
cntMisp = 0
with open(f"{DIR}/MST_wisesight_train_fasttext_training.txt", "w", encoding="utf-8") as fout:
    for sent in wisesight:
      s = []
      for t in sent["tokenized"]:
        s.append(t)
        w = norm_word(t)
        cntToken += 1
        
        if "<lol>" in w:
          s.append("<lol>")
          cntMisp += 1 
        elif "<rep>" in w:
          s.append("<rep>")
          cntMisp += 1
        else:
          if w in MD:
            corr, mint = MD[w]
            if mint:
              s.append("<int>")
            else:
              s.append("<msp>")
            
            cntMisp += 1

      s = " ".join(s)
      # if "<lol>" in s:
      #   print(s)
      #   break
      fout.write(s+"\n")

print()
print("#Tokens:", cntToken)
print("#Misspelling Tokens:", cntMisp, cntMisp*100/cntToken)


#Tokens: 608957
#Misspelling Tokens: 32419 5.323692805895983


# Train FastText

In [52]:
ls Datasets/*_fasttext_training.txt

Datasets/corr_VISTEC-TP-TH-2021_fasttext_training.txt
Datasets/misp_VISTEC-TP-TH-2021_fasttext_training.txt
Datasets/MST_VISTEC-TP-TH-2021_fasttext_training.txt
Datasets/MST_wisesight_train_fasttext_training.txt
Datasets/wisesight_train_fasttext_training.txt


In [53]:
# unsupervised_default = {
#     'model': "skipgram",
#     'lr': 0.05,
#     'dim': 100,
#     'ws': 5,
#     'epoch': 5,
#     'minCount': 5,
#     'minCountLabel': 0,
#     'minn': 3,
#     'maxn': 6,
#     'neg': 5,
#     'wordNgrams': 1,
#     'loss': "ns",
#     'bucket': 2000000,
#     'thread': multiprocessing.cpu_count() - 1,
#     'lrUpdateRate': 100,
#     't': 1e-4,
#     'label': "__label__",
#     'verbose': 2,
#     'pretrainedVectors': "",
#     'seed': 0,
#     'autotuneValidationFile': "",
#     'autotuneMetric': "f1",
#     'autotunePredictions': 1,
#     'autotuneDuration': 60 * 5,  # 5 minutes
#     'autotuneModelSize': ""
# }

In [21]:
trainingDataFile = f"{DIR}/misp_VISTEC-TP-TH-2021_fasttext_training.txt"
model = fasttext.train_unsupervised(trainingDataFile, "cbow", neg=10)
for vec in model.get_nearest_neighbors("แมว"):
  print(vec)
model.save_model(f"{DIR}/../Models/fasttext_misp_VISTEC-TP-TH-2021.bin")

(0.835712194442749, 'หมา')
(0.7926109433174133, 'แมพ')
(0.7894583344459534, 'ขี้หมา')
(0.7862657308578491, 'ลูกอม')
(0.7862566709518433, 'แมค')
(0.7780048251152039, 'ลูกหมา')
(0.7735149264335632, 'ขี้อ้อน')
(0.772652804851532, 'เจ้าก้อน')
(0.7717746496200562, 'แมะ')
(0.7669620513916016, 'ลูกหมู')


In [22]:
trainingDataFile = f"{DIR}/corr_VISTEC-TP-TH-2021_fasttext_training.txt"
model = fasttext.train_unsupervised(trainingDataFile, "cbow", neg=10)
for vec in model.get_nearest_neighbors("แมว"):
  print(vec)
model.save_model(f"{DIR}/../Models/fasttext_corr_VISTEC-TP-TH-2021.bin")

(0.8388424515724182, 'หมา')
(0.7980322241783142, 'แมป')
(0.7958469390869141, 'ลูกอม')
(0.7854677438735962, 'เมี้ยว')
(0.7816352248191833, 'แม็ค')
(0.7790215015411377, 'ขี้หมา')
(0.7773597836494446, 'ขี้อ้อน')
(0.7754426002502441, 'แมะ')
(0.7616896629333496, 'แม่เลี้ยง')
(0.7584441304206848, 'แม็ป')


In [23]:
trainingDataFile = f"{DIR}/MST_VISTEC-TP-TH-2021_fasttext_training.txt"
model = fasttext.train_unsupervised(trainingDataFile, "cbow", neg=10)
for vec in model.get_nearest_neighbors("แมว"):
  print(vec)
model.save_model(f"{DIR}/../Models/fasttext_MST_VISTEC-TP-TH-2021.bin")

(0.8335064649581909, 'หมา')
(0.8030588030815125, 'ขี้หมา')
(0.8010830879211426, 'แมะ')
(0.7994157671928406, 'หมี')
(0.7897553443908691, 'ลูกอม')
(0.7893834114074707, 'แมค')
(0.7860163450241089, 'เปรี้ยง')
(0.7833449244499207, 'ลูกหมู')
(0.7804890871047974, 'แมพ')
(0.779085099697113, 'ลูกหมา')


In [24]:
trainingDataFile = f"{DIR}/wisesight_train_fasttext_training.txt"
model = fasttext.train_unsupervised(trainingDataFile, "cbow", neg=10)
for vec in model.get_nearest_neighbors("แมว"):
  print(vec)
model.save_model(f"{DIR}/../Models/fasttext_wisesight_train.bin")

(0.9966371059417725, 'แมก')
(0.9943579435348511, 'ผี')
(0.9921287298202515, 'แม๊ก')
(0.9912964701652527, 'ตืด')
(0.9906294941902161, 'ศพ')
(0.989380955696106, 'พ่อแม่')
(0.9886942505836487, 'ดิบ')
(0.9884825348854065, 'ตีน')
(0.9882953763008118, 'เเพ้')
(0.9879016876220703, 'ไก่')


In [25]:
trainingDataFile = f"{DIR}/MST_wisesight_train_fasttext_training.txt"
model = fasttext.train_unsupervised(trainingDataFile, "cbow", neg=10)
for vec in model.get_nearest_neighbors("แมว"):
  print(vec)
model.save_model(f"{DIR}/../Models/fasttext_MST_wisesight_train.bin")

(0.9908960461616516, 'ขำ')
(0.9903230667114258, 'ตืด')
(0.9897900223731995, 'แมก')
(0.9892303943634033, 'ยิบ')
(0.9887956976890564, 'จิ')
(0.987443745136261, 'ผี')
(0.9874346256256104, 'แหม')
(0.987338125705719, 'ดิบ')
(0.9873113632202148, 'เคี้ยว')
(0.986656904220581, 'ไก่')


In [26]:
model["แมว"].size

100